# subtitle_analysis
We can extract a film's English-language subtitle track to get the ground-truth dialogue. We can glean clues about a scene's location, characters, and context. While we're not yet ready to use subtitles to analyze the film's entire plot, we can start small and see what localized information we can learn.

We'll be using the `pysrt` library to parse .srt subtitle files and the `spaCy` library for NLP analysis.

In [2]:
import pysrt
import spacy

In [3]:
subs = pysrt.open('../subtitles/booksmart.srt')

In [4]:
len(subs)

2373

Since each two-line dialogue in a subtitle file is explicitly numbered, starting at 1, there's an off-by-one discrepency with the list object (starting at 0). We can offset the list by just duplicating the first subtitle item.

In [5]:
# subtitle files (.srt) are explicitly numbered, and start at 1
subs.insert(0, subs[0])